In [1]:
import stac_mjx 
from pathlib import Path
import mediapy as media
        
# Use glfw or egl for faster rendering if your system supports it
%env MUJOCO_GL=osmesa 
%env PYOPENGL_PLATFORM=osmesa

# Choose parent directory as base path for data files
base_path = Path.cwd().parent

Failed to import warp: No module named 'warp'
Failed to import mujoco_warp: No module named 'warp'
env: MUJOCO_GL=osmesa
env: PYOPENGL_PLATFORM=osmesa


### Load configs

In [2]:
cfg = stac_mjx.load_configs(base_path / "configs")
cfg

Config loaded and validated.


{'stac': {'fit_offsets_path': 'demo_fit_offsets.h5', 'ik_only_path': 'demo_ik_only.h5', 'data_path': 'tests/data/test_rodent_mocap_1000_frames.mat', 'n_fit_frames': 10, 'skip_fit_offsets': False, 'skip_ik_only': True, 'infer_qvels': True, 'n_frames_per_clip': 250, 'mujoco': {'solver': 'newton', 'iterations': 1, 'ls_iterations': 4}}, 'model': {'MJCF_PATH': 'models/rodent.xml', 'FTOL': 0.0001, 'ROOT_FTOL': 1e-05, 'LIMB_FTOL': 1e-06, 'N_ITERS': 6, 'N_ITER_Q': 400, 'N_ITER_M': 2000, 'KP_NAMES': ['Snout', 'EarL', 'EarR', 'SpineF', 'SpineM', 'SpineL', 'TailBase', 'ShoulderL', 'ElbowL', 'WristL', 'HandL', 'ShoulderR', 'ElbowR', 'WristR', 'HandR', 'HipL', 'KneeL', 'AnkleL', 'FootL', 'HipR', 'KneeR', 'AnkleR', 'FootR'], 'KEYPOINT_MODEL_PAIRS': {'AnkleL': 'lower_leg_L', 'AnkleR': 'lower_leg_R', 'EarL': 'skull', 'EarR': 'skull', 'ElbowL': 'upper_arm_L', 'ElbowR': 'upper_arm_R', 'FootL': 'foot_L', 'FootR': 'foot_R', 'HandL': 'hand_L', 'HandR': 'hand_R', 'HipL': 'pelvis', 'HipR': 'pelvis', 'KneeL':

### Prepare your data

When using your own data, make sure the `kp_data` array is of the shape (n_frames, 3 * n_keypoints); note that the (xyz) axis is flattened into the keypoint dimension. `sorted_kp_names` contains an ordered list of each keypoint's name, as used in the config's `model.KEYPOINT_MODEL_PAIRS` field.

In [3]:
kp_data, sorted_kp_names = stac_mjx.load_mocap(cfg, base_path)

### Run stac!

In [4]:
fit_path, ik_only_path = stac_mjx.run_stac(
    cfg,
    kp_data, 
    sorted_kp_names, 
    base_path=base_path
)

Running fit. Mocap data shape: (10, 69)
Root Optimization:
Optimizing first 7 qposes for root
Root optimization finished in 1.47 minutes with an error of 4.3102449126308784e-05
Calibration iteration: 1/6
Pose Optimization:
Pose Optimization finished in 1.29 minutes
Mean: 3.5538312658900395e-05
Standard deviation: 9.482042514719069e-06
starting offset optimization
Begining offset optimization:
Final error of 0.0008094205404631793
Offset optimization finished in 40.960859060287476 seconds
Calibration iteration: 2/6
Pose Optimization:
Pose Optimization finished in 0.02 minutes
Mean: 1.713287019811105e-05
Standard deviation: 5.015708666178398e-06
starting offset optimization
Begining offset optimization:
Final error of 0.0003503651241771877
Offset optimization finished in 38.09789490699768 seconds
Calibration iteration: 3/6
Pose Optimization:
Pose Optimization finished in 0.03 minutes
Mean: 1.8947441276395693e-05
Standard deviation: 9.354230314784218e-06
starting offset optimization
Begini

### Let's visualize!
It's only 10 frames at 50hz, so there's little movement, but you can see that the pose is now aligned to the keypoints!

In [5]:
data_path = base_path / "demo_fit_offsets.h5"
n_frames = 10
save_path = base_path / "videos/direct_render.mp4"

# Call mujoco_viz
cfg, frames = stac_mjx.viz_stac(data_path, 
                                 n_frames, 
                                 save_path, 
                                 start_frame=0, 
                                 camera="close_profile", 
                                 base_path=Path.cwd().parent,
                              )

# Show the video in the notebook (it is also saved to the save_path)
media.show_video(frames, fps=cfg.model.RENDER_FPS)

0it [00:00, ?it/s]/Users/charles/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/subprocess.py:1885: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
10it [00:00, 10.01it/s]


This browser does not support the video tag.
